# Building Machine Learning Classifiers: Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

C:\Users\Justin\anaconda3\envs\harvard\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,21.592042,0.824985,0.346473,0.030705,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977558,0.979354,0.976640,0.967655,0.973046,0.974851,0.004144,1
10,23.873588,0.623889,0.348874,0.048181,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976661,0.978456,0.975741,0.969452,0.971249,0.974312,0.003398,2
6,2.982020,0.105719,0.244947,0.038628,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.980251,0.972172,0.977538,0.970350,0.968553,0.973773,0.004420,3
11,39.277846,0.554663,0.323526,0.016273,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978456,0.976661,0.976640,0.967655,0.969452,0.973773,0.004349,4
3,2.716730,0.329696,0.208442,0.035244,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.979354,0.974865,0.973046,0.968553,0.972147,0.973593,0.003538,5


In [5]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

C:\Users\Justin\anaconda3\envs\harvard\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6,2.563745,0.082841,0.252126,0.022963,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.979354,0.973968,0.975741,0.965858,0.975741,0.974132,0.004493,1
11,44.353751,0.544647,0.366425,0.018711,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.975763,0.974843,0.966757,0.971249,0.973054,0.003646,2
7,26.429613,1.286303,0.454384,0.044181,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978456,0.973070,0.973046,0.966757,0.972147,0.972695,0.003715,3
8,49.845660,0.674995,0.576318,0.038265,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.973070,0.973944,0.966757,0.970350,0.972336,0.003619,4
10,27.778541,0.840164,0.367317,0.073230,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975763,0.974865,0.973046,0.967655,0.969452,0.972156,0.003121,5
